In [ ]:
# Standard libraries
import os

# Third-party libraries
import pandas as pd
import xarray as xr

# Local libraries
from utilities import netcdf


In [ ]:
# Paths
DATA_DIR = os.path.join(os.getcwd(), "data/intermediate/hadukgrid")
OUT_DIR = os.path.join(os.getcwd(), "data/processed/variable")


In [ ]:
NC_FILES = netcdf.list_files(DATA_DIR, path=True)


In [ ]:
names = {
    "groundfrost": "Frost",
    "sun": "Sun",
    "tas": "Temperature",
    "snowLying": "Snow",
    "tasmax": "Maximum temperature",
    "hurs": "Humidity",
    "rainfall": "Rain",
    "sfcWind": "Wind",
    "tasmin": "Minimum temperature",
}

codes = {
    "groundfrost": "frost",
    "sun": "sun",
    "tas": "tas",
    "snowLying": "snow",
    "tasmax": "tasmax",
    "hurs": "hurs",
    "rainfall": "rain",
    "sfcWind": "wind",
    "tasmin": "tasmin",
}


def get_attributes(data_array, to_drop=None):
    """
    Returns a dict describing the attributes of a DataArray
    Optionally, filters the attributes with to_drop
    """
    assert isinstance(to_drop, list) or to_drop is None
    assert isinstance(data_array, xr.DataArray)
    attrs_dict = {}
    for key, value in data_array.attrs.items():
        if key not in to_drop:
            attrs_dict[key] = value
    attrs_dict["id"] = data_array.name
    attrs_dict["name"] = names[data_array.name]
    return attrs_dict


In [ ]:
rows_list = []
for path in NC_FILES:
    with xr.open_dataarray(path, decode_coords="all", chunks="auto") as da:
        attrs_dict = get_attributes(
            da, to_drop=["units", "cell_methods", "level", "standard_name"]
        )
        rows_list.append(attrs_dict)
df = pd.DataFrame(rows_list)
df


In [ ]:
df = (
    df.rename(
        columns={
            "description": "longName",
            "long_name": "description",
            "label_units": "units",
            "plot_label": "label",
        }
    )
    .sort_values("name")
    .reset_index(drop=True)
)

df


In [ ]:
df.to_csv(os.path.join(OUT_DIR, "variable.csv"), index=False)
